# Unsupervised Lab Session

## Learning outcomes:
- Exploratory data analysis and data preparation for model building.
- PCA for dimensionality reduction.
- K-means and Agglomerative Clustering

## Problem Statement
Based on the given marketing campigan dataset, segment the similar customers into suitable clusters. Analyze the clusters and provide your insights to help the organization promote their business.

## Context:
- Customer Personality Analysis is a detailed analysis of a company’s ideal customers. It helps a business to better understand its customers and makes it easier for them to modify products according to the specific needs, behaviors and concerns of different types of customers.
- Customer personality analysis helps a business to modify its product based on its target customers from different types of customer segments. For example, instead of spending money to market a new product to every customer in the company’s database, a company can analyze which customer segment is most likely to buy the product and then market the product only on that particular segment.

## About dataset
- Source: https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis?datasetId=1546318&sortBy=voteCount

### Attribute Information:
- ID: Customer's unique identifier
- Year_Birth: Customer's birth year
- Education: Customer's education level
- Marital_Status: Customer's marital status
- Income: Customer's yearly household income
- Kidhome: Number of children in customer's household
- Teenhome: Number of teenagers in customer's household
- Dt_Customer: Date of customer's enrollment with the company
- Recency: Number of days since customer's last purchase
- Complain: 1 if the customer complained in the last 2 years, 0 otherwise
- MntWines: Amount spent on wine in last 2 years
- MntFruits: Amount spent on fruits in last 2 years
- MntMeatProducts: Amount spent on meat in last 2 years
- MntFishProducts: Amount spent on fish in last 2 years
- MntSweetProducts: Amount spent on sweets in last 2 years
- MntGoldProds: Amount spent on gold in last 2 years
- NumDealsPurchases: Number of purchases made with a discount
- AcceptedCmp1: 1 if customer accepted the offer in the 1st campaign, 0 otherwise
- AcceptedCmp2: 1 if customer accepted the offer in the 2nd campaign, 0 otherwise
- AcceptedCmp3: 1 if customer accepted the offer in the 3rd campaign, 0 otherwise
- AcceptedCmp4: 1 if customer accepted the offer in the 4th campaign, 0 otherwise
- AcceptedCmp5: 1 if customer accepted the offer in the 5th campaign, 0 otherwise
- Response: 1 if customer accepted the offer in the last campaign, 0 otherwise
- NumWebPurchases: Number of purchases made through the company’s website
- NumCatalogPurchases: Number of purchases made using a catalogue
- NumStorePurchases: Number of purchases made directly in stores
- NumWebVisitsMonth: Number of visits to company’s website in the last month

### 1. Import required libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from sklearn.cluster import KMeans
from scipy.stats import zscore
from scipy.spatial import distance
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import PCA

### 2. Load the CSV file (i.e marketing.csv) and display the first 5 rows of the dataframe. Check the shape and info of the dataset.

In [ ]:
df = pd.read_csv('marketing.csv')    
df.head(5)
df.info()


### 3. Check the percentage of missing values? If there is presence of missing values, treat them accordingly.

In [ ]:
df.isnull().sum()/len(df)*100
df['Income'] = df['Income'].fillna(df['Income'].mean())
df.isnull().sum()

### 4. Check if there are any duplicate records in the dataset? If any drop them.

In [ ]:
df.duplicated().sum()

### 5. Drop the columns which you think redundant for the analysis 

In [ ]:
df = df.drop(columns=['ID','Dt_Customer'], axis = 1)

### 6. Check the unique categories in the column 'Marital_Status'
- i) Group categories 'Married', 'Together' as 'relationship'
- ii) Group categories 'Divorced', 'Widow', 'Alone', 'YOLO', and 'Absurd' as 'Single'.

In [ ]:
df['Marital_Status'].value_counts()
df['Marital_Status'] = df['Marital_Status'].replace(['Married', 'Together'], 'relationship')
df['Marital_Status'] = df['Marital_Status'].replace(['Divorced', 'Widow', 'Alone', 'YOLO', 'Absurd'], 'Single')
df['Marital_Status'].value_counts()


### 7. Group the columns 'MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', and 'MntGoldProds' as 'Total_Expenses'

In [ ]:
df['Total_Expenses'] = df['MntWines']+df['MntFruits']+df['MntMeatProducts']+df['MntFishProducts']+df['MntSweetProducts']+df['MntGoldProds']
df.head(3)

### 8. Group the columns 'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', and 'NumDealsPurchases' as 'Num_Total_Purchases'

In [ ]:
df['Num_Total_Purchases'] = df['NumWebPurchases']+df['NumCatalogPurchases']+df['NumStorePurchases']+df['NumDealsPurchases']
df.head(3)

### 9. Group the columns 'Kidhome' and 'Teenhome' as 'Kids'

In [ ]:
df['Kids'] = df['Kidhome']+df['Teenhome']
df.head(3)

### 10. Group columns 'AcceptedCmp1 , 2 , 3 , 4, 5' and 'Response' as 'TotalAcceptedCmp'

In [ ]:
df['TotalAcceptedCmp']= df['AcceptedCmp1']+df['AcceptedCmp2']+df['AcceptedCmp3']+df['AcceptedCmp4']+df['AcceptedCmp5']+df['Response']
df.head(3)

### 11. Drop those columns which we have used above for obtaining new features

In [ ]:
col_dlt = ['AcceptedCmp1','AcceptedCmp2','AcceptedCmp3','AcceptedCmp4','AcceptedCmp5','Response','Kidhome','Teenhome','NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumDealsPurchases','MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
df = df.drop(columns=col_dlt, axis=1)
df.head(3)
df.shape

### 12. Extract 'age' using the column 'Year_Birth' and then drop the column 'Year_birth'

In [ ]:
df['Age'] = 2024 - df['Year_Birth']
df.drop('Year_Birth', axis=1,inplace=True)
df.head()
df_c = df.copy()

### 13. Encode the categorical variables in the dataset

In [ ]:
cat_e = ['Education','Marital_Status']
lbl_encoder = LabelEncoder()
for i in cat_e:
    df[i] = df[[i]].apply(lbl_encoder.fit_transform)
df.head(3)

### 14. Standardize the columns, so that values are in a particular range

In [ ]:
df_1 = df.copy()
scaled_features = StandardScaler().fit_transform(df_1.values)
scaled_features_df = pd.DataFrame(scaled_features, index=df_1.index, columns=df_1.columns)
scaled_features_df.head()


### 15. Apply PCA on the above dataset and determine the number of PCA components to be used so that 90-95% of the variance in data is explained by the same.

In [ ]:
cov_matrix = np.cov(scaled_features.T)
eig_vals, eig_vectors = np.linalg.eig(cov_matrix)
total = sum(eig_vals)
var_exp = [(i/total)*100 for i in sorted(eig_vals, reverse=True)]
cum_car_exp = np.cumsum(var_exp)
plt.bar(range(11),var_exp)
plt.step(range(11),cum_car_exp)
plt.xlabel('Principal component')
plt.ylabel('Explained Variance ratio')

### 16. Apply K-means clustering and segment the data (Use PCA transformed data for clustering)

In [ ]:
pca = PCA(n_components=8)
pca_df = pd.DataFrame(pca.fit_transform(scaled_features_df), columns=['PC1','PC2','PC3','PC4','PC5','PC6','PC7','PC8'])
pca_df.head()
cluster_errors = []
cluster_range = range(2,15)
for num_clusters in cluster_range:
    cluster = KMeans(num_clusters, random_state=100)
    cluster.fit(pca_df)
    cluster_errors.append(cluster.inertia_)
cluster_df = pd.DataFrame({'num_clusters': cluster_range,'cluster_errors': cluster_errors})
plt.plot(cluster_df['num_clusters'],cluster_df['cluster_errors'], marker='o')
kmeans = KMeans(n_clusters=3,random_state=100)
kmeans.fit(pca_df)
label = pd.DataFrame(kmeans.labels_,columns=['Label'])
kmeans_df = pca_df.join(label)
kmeans_df.head()
kmeans_df['Label'].value_counts()
sns.scatterplot(data=kmeans_df, x='PC1', y='PC2', hue='Label', palette='viridis')


### 17. Apply Agglomerative clustering and segment the data (Use Original data for clustering), and perform cluster analysis by doing bivariate analysis between the cluster label and different features and write your observations.

In [ ]:
merg = linkage(scaled_features, method='ward')
dendrogram(merg, leaf_rotation=90)
plt.show()
from sklearn.metrics import silhouette_score
for i in range(2,25):
    hier = AgglomerativeClustering(n_clusters=i)
    hier = hier.fit(scaled_features_df) 
    labels = hier.fit_predict(scaled_features_df)
    print(i, silhouette_score(scaled_features_df, labels))
    hie_cluster = AgglomerativeClustering(n_clusters=3, linkage='ward')
hie_cluster_model = hie_cluster.fit(scaled_features_df)
df_label1 = pd.DataFrame(hie_cluster_model.labels_, columns=['Labels'])
df_label1.head()
df_hier = df.join(df_label1)
df_hier.head()

### Visualization and Interpretation of results

In [ ]:
1.Visualization Techniques: Utilize scatter plots, heatmaps, and dendrograms to visualize clusters, providing clarity and insight into segmentation.
2.Comparative Visualizations: Create visualizations to compare clusters, highlighting differences in demographics, purchasing behavior, and
response to marketing campaigns.
3.Cluster Characteristics Analysis: Analyze each cluster's characteristics to understand distinct customer segments, identifying key attributes
or behaviors that define them.
4.Identify Key Attributes: Determine the most influential variables for cluster membership, illustrating them across clusters using radar charts
or parallel coordinate plots.
5.Comparison Across Dimensions: Compare clusters across various dimensions such as response to marketing campaigns, customer satisfaction, 
and loyalty metrics to gain comprehensive insights.

-----
## Happy Learning
-----